In [1]:
%gui qt5
import numpy as np
from skimage import io
from starfish import Experiment, display, Codebook, ExpressionMatrix, IntensityTable
from starfish.core.segmentation_mask import SegmentationMaskCollection
from starfish.image import Filter
from starfish.spots import DetectSpots, AssignTargets
from starfish.types import Axes, Coordinates, Features, FunctionSource

In [2]:
# Load the data
exp = Experiment.from_json("./output/experiment.json")
im = exp.fov().get_image('primary')

In [3]:
# View the image stack
display(im)

100%|██████████| 21/21 [00:08<00:00,  2.61it/s]


In [3]:
# Filter the image
ghp = Filter.GaussianHighPass(sigma=3)
high_passed = ghp.run(im, verbose=True, in_place=False)


glp = Filter.GaussianLowPass(sigma=1)
low_passed = glp.run(high_passed, in_place=False, verbose=True)

100%|██████████| 21/21 [00:00<00:00, 1088.66it/s]


In [4]:
# Max intensity project the spots spots image
mproj = Filter.Reduce((Axes.ZPLANE,), func='max', module=FunctionSource.np)
mip = mproj.run(low_passed)

In [15]:
# Find the spots
p = DetectSpots.BlobDetector(
    min_sigma=1,
    max_sigma=10,
    num_sigma=10,
    threshold=0.00001,
    measurement_type='mean',
)

# blobs = dots; define the spots in the dots image, but then find them again in the stack.
intensities = p.run(mip)

In [16]:
display(low_passed, intensities)

In [ ]:
from InSituToolkit.analysis import save_stack

# Save the nuclei stack
nuc = exp.fov().get_image('nuclei')

nuc_mip = mproj.run(nuc)

save_stack(nuc, './nuc.tif')